[View in Colaboratory](https://colab.research.google.com/github/gowtham91m/gofundme/blob/master/gofundme.ipynb)

In [0]:
%%capture
!pip install "requests[security]"

In [0]:
from bs4 import BeautifulSoup as bs
import requests
from IPython.display import display, clear_output
import re
import pandas as pd
from time import time
from collections import defaultdict

In [0]:
# all categories and the cid
class web_scraper:
  def __init__(self,url):
    self.url = url
    
  def get_categories(self):
    soup = requests.get(url)
    soup = bs(soup.text,'html.parser')
    category = soup.findAll(class_='text-black')
    categories = [i.text for i in category]  
    return categories[:1] #16
  
  def details_parser(self,url):
    try:
      soup=bs(requests.get(url).text,'html.parser')
      text = soup.findAll(class_="co-story truncate-text truncate-text--description js-truncate")[0].text.strip()
    except:
      text = 'exception occured for' + url
    return({'text':text})
  
  def scrape(self):
    df = pd.DataFrame({})
    for i in self.get_categories():
      print(i,end='  ')
      i='-'.join(i.split(' '))
      i = 'animal' if i == 'Animals' else i
      url = 'https://www.gofundme.com/discover/'+i+'-fundraiser'
      soup = bs(requests.get(url).text,'html.parser')
      cid = re.findall('\d+',re.findall('cid=\'\s\+\s\'\d+', soup.find_all('script')[13].text)[0])[0]
      Resp,page = True,1
      while Resp:
        url = 'https://www.gofundme.com/mvc.php?route=categorypages/load_more&page='+str(page)+'&term=&cid='+cid
        soup = requests.get(url)
        soup = bs(soup.text, 'html.parser')
        if len(soup) <1: Resp =False
        name = [ hit.text  for hit in soup.findAll(attrs={'class' : 'fund-title truncate-single-line show-for-medium'})]
        href = [i['href'] for i in soup.findAll('a',attrs={'class':'campaign-tile-img--contain'})]
        location = [i.text[1:-1] for i in soup.findAll(class_='fund-item fund-location truncate-single-line')]
        raised = [i.findAll('strong')[0].text[:-7] for i in soup.findAll(class_="fund-item truncate-single-line")]
        goal = [re.findall('\$\d+\,\d+',i.text)[1] for i in soup.findAll(class_="fund-item truncate-single-line")]
        
        details =defaultdict(list)
        for link in soup.findAll('a',attrs={'class':'campaign-tile-img--contain'}):
          for key, value in self.details_parser(link['href']).items():
            details[key].append(value)
        
        df = df.append(pd.DataFrame({'category':[i]*len(name),
                                     'name':name,
                                     'href':href,
                                     'location':location, 
                                     'raised':raised,
                                     'goal':goal,
                                     'text':details['text']}))
        if (page%10==0):print(page,end=' ')
        page+=1
      print('\n')
    clear_output()
    columns = ['category','name','href','location','raised','goal','text']
    return df[columns]

In [282]:
url = 'https://www.gofundme.com/discover'
start_time = time()
df = web_scraper(url).scrape()
print((time()-start_time)/60)

7.8066372315088906


In [0]:
#df.head()